# ATSC-411 - SYNOPTIC METEOROLOGY | PV LAB

<br>
<br>
<br>

**************
<br>
<br>
<br>
<br>
<br>
<br>


## IMPORT LIBRARIES


In [ ]:
# import packages
import numpy as np                               # numpy for math & number operations
import cartopy.crs as ccrs                       # cartopy coordinate refrence system for mapping -- to turn a matplotlib axis into a geoaxis (map)
import cartopy.feature as cfeature               # cartopy features (borders, states, oceans, rivers, etc) for mapping -- add map components to a map
import matplotlib.pyplot as plt                  # matplotlib for plotting figures, axes, graphs, etc
from matplotlib.colors import LinearSegmentedColormap

import helper_functions as helper
import metpy.calc as mpcalc
from metpy.units import units
from metpy.interpolate import cross_section
from metpy.interpolate import interpolate_to_isosurface

print("[+] packages imported...")

**************
<br>
<br>
<br>
<br>
<br>
<br>

## LOAD RAP REANALYSIS DATA FOR A GIVEN DATE

In [ ]:
# date/time components
year  = "2024"
month = "10"
day   = "23"
hour  = "00"


# load an `xarray` dataset of RAP reanalysis data
# this will take a few seconds to load
rap_data = helper.give_me_rap_data(year, month, day, hour)


print("\n[+] data load complete...")

**************
<br>
<br>
<br>
<br>
<br>
<br>

## Compute plan-view map variables
#### Compute potential temperature, lat/lon grid deltas, potential vorticity, theta and wind on a 2PVU isosurface and geopotential hgt slices

In [ ]:
#################################
# COMPUTE THETA AND PV
################################# 


rap_data['wind_speed'] = mpcalc.wind_speed(rap_data['u-component_of_wind_isobaric'], rap_data['v-component_of_wind_isobaric'])

# use metpy to compute theta & add it into the `rap-data` DataSet
rap_data['theta'] = mpcalc.potential_temperature(rap_data['isobaric'],rap_data['Temperature_isobaric'])

# use metpy to compute latitude / longitude grid deltas (dx, dy) for PV calculation
dx, dy = mpcalc.lat_lon_grid_deltas(rap_data['lon'].values, rap_data['lat'].values)

# use metpy to compute PV and add it into the `rap_data` DataSet
rap_data['pv'] = mpcalc.potential_vorticity_baroclinic(rap_data['theta'],
                                                        rap_data['isobaric'],
                                                        u=rap_data['u-component_of_wind_isobaric'],
                                                        v=rap_data['v-component_of_wind_isobaric'],
                                                        dx=dx[None, None, :, :], dy=dy[None, None, :, :],
                                                        latitude=rap_data['lat'])

thta_on_2pvu = interpolate_to_isosurface(rap_data['pv'].values, rap_data['theta'].values,  2*1e-6, bottom_up_search=True)
u_on_2pvu    = interpolate_to_isosurface(rap_data['pv'][:,0,:,:].values, rap_data['u-component_of_wind_isobaric'][0,:,:,:].values, 2*1e-6, bottom_up_search=True)
v_on_2pvu    = interpolate_to_isosurface(rap_data['pv'][:,0,:,:].values, rap_data['v-component_of_wind_isobaric'][0,:,:,:].values, 2*1e-6, bottom_up_search=True)

#################################
# DEFINE USEFUL PRESSURE LEVEL SLICES
################################# 
# create variables that hold the "index" (location) of a given height in the isobaric dimension
# i.e., 300 hPa is the 8th value in the isobaric dimension, use it to slice other variables
plev250 = list(rap_data['isobaric']).index(((250 * units('hPa')).to(rap_data['isobaric'].units)).m)
plev300 = list(rap_data['isobaric']).index(((300 * units('hPa')).to(rap_data['isobaric'].units)).m)
plev500 = list(rap_data['isobaric']).index(((500 * units('hPa')).to(rap_data['isobaric'].units)).m)


#################################
# CREATE A TIDY DATE STRING
################################# 
try:
    data_date = rap_data['time'].values[0]
except:
    data_date = rap_data['time1'].values[0]
    pass
valid_date = f'{data_date}'

print("\n[+] plan view data preparation complete...")

**************
<br>
<br>
<br>
<br>
<br>
<br>

## Define the `build_map()`, & `build_inset_map()` functions & create custom colormaps

In [ ]:
#--------------------------------------------------------------------------------------
# BUILD MAPS FUNCTION -----------------------------------------------------------------
#--------------------------------------------------------------------------------------
# define build maps func
def build_map(extent=[-121, -73, 21, 56]):
    
    fig = plt.figure(figsize=(16, 12))
    ax = plt.axes(projection=ccrs.LambertConformal())

    # apply the map extent (lat/lon bounding box)
    ax.set_extent(extent)
    # axis aspect ratio
    ax.set_box_aspect(0.7)
    # add map features
    ax.add_feature(cfeature.STATES, edgecolor='navy', alpha=0.5, linestyle='-', linewidth=1, zorder=10)
    ax.add_feature(cfeature.BORDERS, color='navy', alpha=1, linestyle='-', linewidth=1, zorder=11)
    ax.add_feature(cfeature.COASTLINE, color='navy', alpha=0.5, linestyle='-', linewidth=1, zorder=11)

    # apply tight layout to the figure (keeps things tiddy)
    plt.tight_layout()

    # return the figure axis
    return fig, ax


#--------------------------------------------------------------------------------------
# BUILD INSET MAP FUNCTION -----------------------------------------------------------------
#--------------------------------------------------------------------------------------
def build_inset_map(extent=[-121, -73, 21, 56]):

    # SET UP MAP INSET AXIS
    proj = ccrs.LambertConformal()
    ax = fig.add_axes([0.73, 0.712, 0.17, 0.17], projection=proj)
    ax.set_box_aspect(0.7)
    ax.set_extent(extent, ccrs.PlateCarree())

    ax.add_feature(cfeature.COASTLINE, edgecolor='navy', linewidth=0.5)
    ax.add_feature(cfeature.STATES, edgecolor='navy', linewidth=0.5)

    return ax




#--------------------------------------------------------------------------------------
# DEFINE COLORMAPS    -----------------------------------------------------------------
#--------------------------------------------------------------------------------------

# wind speed colormap
wdsp_colors = [(0.000, '#FFFFFF'),(0.004, '#87CEFA'),(0.166, '#6A5ACD'),(0.250, '#E696DC'),(0.333, '#C85ABE'),
          (0.416, '#A01496'), (0.500, '#C80028'), (0.583, '#DC283C'), (0.666, '#F05050'),(0.750, '#FAF064'),
          (0.833, '#DCBE46'), (0.916, '#BE8C28'),(1.000, '#A05A0A')]
wdsp_cmap = LinearSegmentedColormap.from_list("custom_cmap",wdsp_colors,N=256)

# potential vorticity colormap
pv_clevs  = np.append(np.arange(-1.4, 2, 0.2),np.arange(2,10.2,0.4))
pv_colors = ["blue","lightblue","lightblue","yellow","orange","red","darkred"]
pv_cmap   = LinearSegmentedColormap.from_list("custom_cmap",pv_colors)

# theta on the 2PVU surface colormap
theta_2pvu_colors = ["purple","lightblue","blue","green","yellow","red","darkred"]
theta_2pvu_cmap = LinearSegmentedColormap.from_list("custom_cmap", theta_2pvu_colors)


print("\n[+] functions and colormaps created...")


**************
<br>
<br>
<br>
<br>
<br>
<br>


## 300hPa plan view map of geopotential height and wind

In [ ]:
#################################
# BUILD AXIS AND FIGURE
#################################
fig, ax = build_map()


# define the level to slice
slice = plev300



#################################
# PLOT DATA ON THE MAP
#################################

# plot filled contours of wind speed @ 300hPa
contourf = ax.contourf(rap_data['lon'], rap_data['lat'], rap_data['wind_speed'][0,slice,:,:]* 1.94384, np.arange(40, 160, 5), extend='both',
                 cmap=wdsp_cmap, alpha=0.7, transform=ccrs.PlateCarree(), zorder=4)



# plot geopotential height contours
contour = ax.contour(rap_data['lon'], rap_data['lat'], rap_data['Geopotential_height_isobaric'][0,slice,:,:], np.arange(0, 12000, 60),
                colors='black', linewidths=3.0, linestyles='-',
                transform=ccrs.PlateCarree(), zorder=11)
plt.clabel(contour, fontsize=8, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)


# plot wind barbs
every = 20
barbs = ax.barbs(rap_data['lon'].values[0::every, 0::every], rap_data['lat'].values[0::every, 0::every],
                 rap_data['u-component_of_wind_isobaric'][0,slice,:,:].values[0::every, 0::every]* 1.94384, 
                 rap_data['v-component_of_wind_isobaric'][0,slice,:,:].values[0::every, 0::every]* 1.94384,
                 length=6.5, alpha=0.7, transform=ccrs.PlateCarree(), zorder=12)


# add some plot titles
plt.title('  300-hPa GFS 300-hPa Geopotential Height (dam),\n'
          '  and Wind Barbs (kt)', loc='left', fontsize=20)
plt.title(f'{valid_date[0:10]} {valid_date[11:-13]}UTC  ', loc='right', fontsize=20)


# colorbar for filled contour
cbar = plt.colorbar(contourf, aspect=70, fraction=0.02, ax=ax, orientation='horizontal', pad=-0.01, extendrect=True)
cbar.set_label(r'Wind Speed (kts)', fontsize=15)

print("\n[+] plotting...")

# Plot a simple 300hPa PV map
### Geopotential Height, Wind, Potential Vorticity

In [ ]:
#################################
# BUILD AXIS AND FIGURE
#################################
fig, ax = build_map()


# define the level to slice
slice = plev300



#################################
# PLOT DATA ON THE MAP
#################################



#################################
# PLOT DATA ON THE MAP
#################################
contourf = ax.contourf(rap_data['lon'], rap_data['lat'], rap_data['pv'][slice,0,:,:]*1e6, pv_clevs, cmap=pv_cmap,
                 transform=ccrs.PlateCarree(),extend='both')

# plot a single dashed contour @ 2PVU
pv_contour = ax.contour(rap_data['lon'], rap_data['lat'], rap_data['pv'][slice,0,:,:]*1e6, [2], colors='navy',linestyles='dashed',linewidths=2,
                 transform=ccrs.PlateCarree())


# plot geopotential height contours
contour = ax.contour(rap_data['lon'], rap_data['lat'], rap_data['Geopotential_height_isobaric'][0,slice,:,:], np.arange(0, 12000, 60),
                colors='black', linewidths=3.0, linestyles='-',
                transform=ccrs.PlateCarree(), zorder=11)
plt.clabel(contour, fontsize=8, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)


# plot wind barbs
every = 20
barbs = ax.barbs(rap_data['lon'].values[0::every, 0::every], rap_data['lat'].values[0::every, 0::every],
                 rap_data['u-component_of_wind_isobaric'][0,slice,:,:].values[0::every, 0::every]* 1.94384, 
                 rap_data['v-component_of_wind_isobaric'][0,slice,:,:].values[0::every, 0::every]* 1.94384,
                 length=6.5, alpha=0.7, transform=ccrs.PlateCarree(), zorder=12)


# add some plot titles
plt.title('  300-hPa GFS PV (PVU), 300-hPa Geopotential Height (dam),\n'
          '  and Wind Barbs (kt)', loc='left', fontsize=20)
plt.title(f'{valid_date[0:10]} {valid_date[11:-13]}UTC  ', loc='right', fontsize=20)

# colorbar for filled contour
cbar = plt.colorbar(contourf, aspect=70, fraction=0.02, ax=ax, orientation='horizontal', pad=-0.01, extendrect=True)
cbar.set_label(r'Potential Vorticity Units (PVU; $\rm{10^{-6}\ K\ kg^{-1}\ m^{2}\ s^{-1}})$' + ' | 2PVU (dashed)', fontsize=15)


print("\n[+] plotting...")

**************
<br>
<br>
<br>
<br>
<br>
<br>

# **CROSS SECTION ANALYSIS**
<br>

#### 1. Use MetPy's `cross_section` to create variables of "sliced" cross section data
#### 2. Plot cross section analyses of PV and static stability

<br>
<br>

## 1. Compute Cross Section Data 

In [ ]:
#################################
# COMPUTE CROSS SECTION USING METPY
#################################
# define the start and end point of the cross section
start = (35, -125.0)
end = (40, -80.0)

# use metpy's `cross_section` to create a new DataSet of cross section "2D" data
cs_data = cross_section(rap_data.metpy.parse_cf().squeeze(), start, end).set_coords(('lat', 'lon'))

# using the cross section DataSet, calculate the normal component of the wind to the cross section plane
cs_data['nwind'] = mpcalc.normal_component(cs_data['u-component_of_wind_isobaric'], cs_data['v-component_of_wind_isobaric'])

# using metpy compute static stability 
static_stability = mpcalc.first_derivative(cs_data['theta'], axis=0, x=cs_data['isobaric'])

print("\n[+] data preparation complete...")


**************
<br>
<br>
<br>
<br>
<br>
<br>

## 2a. PV Cross Section

In [ ]:
#################################
# CREATE SIMPLE FIGURE
#################################
fig = plt.figure(1, figsize=(14, 10))
ax = plt.subplot(111)


#################################
# PLOT CROSS SECTION DATA
#################################
# plot contourf of PV
pv_contourf = plt.contourf(cs_data['lon'], cs_data['isobaric']/100, cs_data['pv']*1e6, pv_clevs, extend='both', cmap=pv_cmap)

# plot contour of normal-wind to cross section plane, add labels
nwnd_contour = plt.contour(cs_data['lon'], cs_data['isobaric']/100, cs_data['nwind']* 1.94384 , colors='black',linewidths=2)
labels = plt.clabel(nwnd_contour, nwnd_contour.levels[::2], fontsize=10, inline=True,fmt = '%1.0f')
for l in labels:
    l.set_rotation(0)

# plot 2PVU contour
plt.contour(cs_data['lon'], cs_data['isobaric']/100, cs_data['pv']*1e6, [2],
            colors='magenta',linewidths=3,linestyles='dashed')

# plot theta contours
plt.contour(cs_data['lon'], cs_data['isobaric']/100, cs_data['theta'], np.arange(280,500,10),
            colors='white',linewidths=1)


#################################
# MANIPULATE PLOT AXIS, ADD GRID
#################################
ax.grid(True)
ax.set_yscale('symlog')
ax.set_ylim([1000,100])
ax.set_yticklabels(np.arange(1000, 50, -100))
ax.set_yticks(np.arange(1000, 50, -100))

#################################
# ADD PLOT AXIS LABELS AND TITLES
#################################
plt.ylabel('Pressure (hPa)')
plt.xlabel(r'Longitude $\rm{(^\circ E)}$')
plt.title('  RAP Vertical Cross Section\n  Potential Vorticity (PVU), Normal Wind-Component (kts), Potential Temperature (K)', loc='left', fontsize=14)
plt.title(f'{valid_date[0:10]} {valid_date[11:-13]}UTC  ', loc='right', fontsize=14)

#################################
# ADD FILLED CONTOUR COLORBAR
#################################
cbar = plt.colorbar(pv_contourf, aspect=70, fraction=0.02, ax=ax, orientation='horizontal', pad=0.07, extendrect=True)
cbar.set_label(r'Potential Vorticity Units (PVU; $\rm{10^{-6}\ K\ kg^{-1}\ m^{2}\ s^{-1}})$', fontsize=10)




#################################
# CREATE A 2ND AXIS TO PLOT TERRAIN
#################################
# Create the second axis, hide it, set y limit
# plot sfc hghts - 1000hPa hght for a 1000hPa-100hPa yaxis 
ax2 = ax.twinx()
ax2.get_yaxis().set_visible(False)   
ax2.spines['right'].set_visible(False)
ax2.set_ylim([0,15000])
ax2.plot(cs_data['lon'], cs_data['Geopotential_height_surface'] - cs_data['Geopotential_height_isobaric'][-1], color='saddlebrown')
ax2.fill_between(cs_data['lon'], 0, cs_data['Geopotential_height_surface']- cs_data['Geopotential_height_isobaric'][-1], color='saddlebrown')




#################################
# BUILD AND PLOT MAP INSET
#################################
inset_ax = build_inset_map()
# define map projection
inset_projection = ccrs.LambertConformal()

# PLOT CROSS SECTION POINTS
endpoints = inset_projection.transform_points(ccrs.Geodetic(),*np.vstack([start, end]).transpose()[::-1])
inset_ax.scatter(endpoints[:, 0], endpoints[:, 1], c='k', zorder=2)
inset_ax.plot(endpoints[:,0], endpoints[:,1], c='k', zorder=2, lw=2)

# PLOT 300hPa GEOPOTENTIAL HEIGHTS CONTOUR
cs = inset_ax.contour(rap_data['lon'], rap_data['lat'], rap_data['Geopotential_height_isobaric'][0, plev300, :, :], np.arange(0, 18000, 60),
                      colors='black',transform=ccrs.PlateCarree(), linewidths=1)

# PLOT 300hPa PV CONTOURF
inset_contourf = inset_ax.contourf(rap_data['lon'], rap_data['lat'], rap_data['pv'][plev300, 0, :, :]*1e6, pv_clevs, cmap=pv_cmap,
                                   transform=ccrs.PlateCarree(), extend='both')

print("\n[+] plotting...")


**************
<br>
<br>
<br>
<br>
<br>
<br>

## 2b. Static stability cross section

In [ ]:
#################################
# CREATE SIMPLE FIGURE
#################################
fig = plt.figure(1, figsize=(14, 10))
ax = plt.subplot(111)


#################################
# PLOT CROSS SECTION DATA
#################################
# plot static stability as a filled contour
static_stability_contourf = plt.contourf(cs_data['lon'], cs_data['isobaric']/100, static_stability*-1,
                                         np.arange(0, 0.015, 0.0005), extend='both', cmap='brg')
for l in labels:
    l.set_rotation(0)


# plot 2PVU contour
plt.contour(cs_data['lon'], cs_data['isobaric']/100, cs_data['pv']*1e6, [2],
            colors='magenta',linewidths=3,linestyles='dashed')

# plot theta contours
plt.contour(cs_data['lon'], cs_data['isobaric']/100, cs_data['theta'], np.arange(280,500,10),
            colors='white',linewidths=1)




#################################
# MANIPULATE PLOT AXIS, ADD GRID
#################################
ax.grid(True)
ax.set_yscale('symlog')
ax.set_ylim([1000,100])
ax.set_yticklabels(np.arange(1000, 50, -100))
ax.set_yticks(np.arange(1000, 50, -100))

#################################
# ADD PLOT AXIS LABELS AND TITLES
#################################
plt.ylabel('Pressure (hPa)')
plt.xlabel(r'Longitude $\rm{(^\circ E)}$')
plt.title('  RAP Vertical Cross Section\n  Static Stability '+r'$\rm{(\frac{K}{hPa})}$'+', Potential Temperature (K), 2PVU', loc='left', fontsize=14)
plt.title(f'{valid_date[0:10]} {valid_date[11:-13]}UTC  ', loc='right', fontsize=14)

#################################
# ADD FILLED CONTOUR COLORBAR
#################################
cbar = plt.colorbar(static_stability_contourf, aspect=70, fraction=0.02, ax=ax, orientation='horizontal', pad=0.07, extendrect=True)
cbar.set_label(r'$\rm{\frac{\partial \theta}{\partial p}\ (\frac{K}{hPa})}$'+' | 2PVU (dashed)', fontsize=12)






#################################
# CREATE A 2ND AXIS TO PLOT TERRAIN
#################################
# Create the second axis, hide it, set y limit
# plot sfc hghts - 1000hPa hght for a 1000hPa-100hPa yaxis 
ax2 = ax.twinx()
ax2.get_yaxis().set_visible(False)   
ax2.spines['right'].set_visible(False)
ax2.set_ylim([0,15000])
ax2.plot(cs_data['lon'], cs_data['Geopotential_height_surface'] - cs_data['Geopotential_height_isobaric'][-1], color='saddlebrown')
ax2.fill_between(cs_data['lon'], 0, cs_data['Geopotential_height_surface']- cs_data['Geopotential_height_isobaric'][-1], color='saddlebrown')





#################################
# BUILD AND PLOT MAP INSET
#################################
inset_ax = build_inset_map()
# define map projection
inset_projection = ccrs.LambertConformal()

# PLOT CROSS SECTION POINTS
endpoints = inset_projection.transform_points(ccrs.Geodetic(),*np.vstack([start, end]).transpose()[::-1])
inset_ax.scatter(endpoints[:, 0], endpoints[:, 1], c='k', zorder=2)
inset_ax.plot(endpoints[:,0], endpoints[:,1], c='k', zorder=2, lw=2)

# PLOT 300hPa GEOPOTENTIAL HEIGHTS CONTOUR
cs = inset_ax.contour(rap_data['lon'], rap_data['lat'], rap_data['Geopotential_height_isobaric'][0, plev300, :, :], np.arange(0, 18000, 60),
                      colors='black',transform=ccrs.PlateCarree(), linewidths=1)

# PLOT 300hPa PV CONTOURF
inset_contourf = inset_ax.contourf(rap_data['lon'], rap_data['lat'], rap_data['pv'][plev300, 0, :, :]*1e6, pv_clevs, cmap=pv_cmap,
                                   transform=ccrs.PlateCarree(), extend='both')

**************
<br>
<br>
<br>
<br>
<br>
<br>

# **C. ANALYZE DATA ON THE 2PVU ISOSURFACE**
#### 1. Calculate theta, u, & v on the 2PVU surface
#### 2. Plot a plan-view map of the 2PVU surface

<br>
<br>

## 1. Compute variables on the 2PVU isosurface

In [ ]:
# use metpy to compute values of theta, u, & v on an isosurface, where the isosurface is the 2PVU surface
thta_on_2pvu = interpolate_to_isosurface(rap_data['pv'].values, 
                                         rap_data['theta'].values,  2*1e-6, bottom_up_search=True)
u_on_2pvu    = interpolate_to_isosurface(rap_data['pv'][:,0,:,:].values, 
                                         rap_data['u-component_of_wind_isobaric'][0,:,:,:].values, 2*1e-6, bottom_up_search=True)* 1.94384
v_on_2pvu    = interpolate_to_isosurface(rap_data['pv'][:,0,:,:].values, 
                                         rap_data['v-component_of_wind_isobaric'][0,:,:,:].values, 2*1e-6, bottom_up_search=True)* 1.94384

<br>

## 2. Plot a plan-view map of the 2PVU isosurface

In [ ]:
#################################
# BUILD MAP FIGURE
#################################
fig, ax = build_map()

contourf = ax.contourf(rap_data['lon'], rap_data['lat'], thta_on_2pvu, np.arange(280,421,3), cmap=theta_2pvu_cmap,
                 transform=ccrs.PlateCarree(), extend='both',alpha=.5)

# plot wind barbs
every = 20
barbs = ax.barbs(rap_data['lon'].values[0::every, 0::every], rap_data['lat'].values[0::every, 0::every],
                 u_on_2pvu[0::every, 0::every], v_on_2pvu[0::every, 0::every],
                 length=6.5, alpha=0.7, transform=ccrs.PlateCarree(), zorder=12)


# add some plot titles
plt.title('  2-PVU-Isosurface Potential Temperature (K), Wind Barbs (kt)', loc='left', fontsize=20)
plt.title(f'{valid_date[0:10]} {valid_date[11:-13]}UTC  ', loc='right', fontsize=20)

# colorbar for filled contour
cbar = plt.colorbar(contourf, aspect=70, fraction=0.02, ax=ax, orientation='horizontal', pad=-0.01, extendrect=True)
cbar.set_label(r'2-PVU-Isosurface Potential Temperature ($\rm{K}$)', fontsize=15)